# Alice

In [6]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN
from keras.models import Sequential
import numpy as np

from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
%matplotlib inline

fp = open("D:/Value of Dream/Marathon_Github/NLP/data/Alice.txt", 'rb')
lines = []  ## 注意  有加 s
## strip() remove certain characters (such as whitespace) from the left and right parts of strings 抓一列  然後把最前面跟最後面的空白去掉(中間空白留著)
## With lower() we can treat "cat" and "CAT" and "Cat" the same
for line in fp:  ## 從 fp 中  逐列抓資料
    line = line.strip().lower()  
    line = line.decode("ascii", "ignore")   
    ##以 encoding 指定的編碼格式解碼字串  "ignore"  (just leave the character out of the Unicode result)
    if len(line) == 0:  ## 如果抓到的資料  甚麼都沒有就繼續
        continue
    lines.append(line)
fp.close()
text = " ".join(lines)  ##將序列中的元素以指定的字元連接生成一個新的字串  換列  應該就是用空白分隔
print(text[800:3000])


ing nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, and what is the use of a book, thought alice without pictures or conversations? so she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a white rabbit with pink eyes ran close by her. there was nothing so very remarkable in that; nor did alice think it so very much out of the way to hear the rabbit say to itself, oh dear! oh dear! i shall be late! (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, alice started to her feet, for it flashed across her mind that she had never 

In [8]:
# 根據文本內容生成字典
chars = set([c for c in text])  
## 將所有字元切出來 {' ', '!', '#', '$', '%', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '[', ']', '_', 'a', 'b',… 'z'}
nb_chars = len(chars)#總共用了多少字元
char2index = dict((c, i) for i, c in enumerate(chars))
## enumerate() 函數用於將一個可遍歷的資料物件(如清單、元組或字串)組合為一個索引序列，同時列出資料和資料下標，一般用在 for 迴圈當中。
## seasons = ['Spring', 'Summer', 'Fall', 'Winter']    list(enumerate(seasons, start=1))     [(1, 'Spring'), (2, 'Summer'), (3, 'Fall'), (4, 'Winter')]
## dict((c, i) for i, c in enumerate(chars))   將 c 與 i 編成字典  其中 i 是chars 裡面的字元出現順序

index2char = dict((i, c) for i, c in enumerate(chars))
print('字典大小：', nb_chars,'\r\n字典內容：\r\n' ,chars)
print('字典索引：',char2index)
#char2index
print(f'index2char:{index2char}')


字典大小： 55 
字典內容：
 {'o', '3', 'g', 'z', '6', ']', '4', '?', ')', ';', '2', 'c', 'k', 'j', 'u', 's', 'i', 'a', 'h', '5', 'b', 'f', '0', '_', '7', 'l', '[', '(', 'y', 'x', 'r', '!', 't', '1', 'v', 'd', 'm', 'e', ',', '/', '%', '.', ' ', '#', ':', 'w', '$', '@', '*', '9', 'q', '-', 'p', 'n', '8'}
字典索引： {'o': 0, '3': 1, 'g': 2, 'z': 3, '6': 4, ']': 5, '4': 6, '?': 7, ')': 8, ';': 9, '2': 10, 'c': 11, 'k': 12, 'j': 13, 'u': 14, 's': 15, 'i': 16, 'a': 17, 'h': 18, '5': 19, 'b': 20, 'f': 21, '0': 22, '_': 23, '7': 24, 'l': 25, '[': 26, '(': 27, 'y': 28, 'x': 29, 'r': 30, '!': 31, 't': 32, '1': 33, 'v': 34, 'd': 35, 'm': 36, 'e': 37, ',': 38, '/': 39, '%': 40, '.': 41, ' ': 42, '#': 43, ':': 44, 'w': 45, '$': 46, '@': 47, '*': 48, '9': 49, 'q': 50, '-': 51, 'p': 52, 'n': 53, '8': 54}
index2char:{0: 'o', 1: '3', 2: 'g', 3: 'z', 4: '6', 5: ']', 6: '4', 7: '?', 8: ')', 9: ';', 10: '2', 11: 'c', 12: 'k', 13: 'j', 14: 'u', 15: 's', 16: 'i', 17: 'a', 18: 'h', 19: '5', 20: 'b', 21: 'f', 22: '0', 23: '_

In [9]:
#把一篇文章變成一筆一筆資料
SEQLEN = 10  ## 想用前10個字元去預測第 11個字元
STEP = 1
input_chars = []#x有10個
label_chars = []#y有一個
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])  ## 前10個字元當作 x input
    label_chars.append(text[i + SEQLEN])    ## 第11個字元當作 y output

#input_chars
#label_chars
print('input_chars:', input_chars[:100])     ## show 出前 100筆資料
print('\r\nlabel_chars:', label_chars[:100])


# The next step is to vectorize these input and label texts     前面的資料是字元   現在要把xy字元依對照表  轉成數字資料
## 先宣告整個資料的大小矩陣 X (158773(筆), 10(列), 55(行))     y (158773(筆), 55(行))  
## 注意資料的擺放方式  每一個字元  佔一列(55行其中一個為1)  共 10 列(10個字元)  
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)  
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1
    
print('X[0]', X[0]*1)  ## show 出第一筆輸入變數x 資料!! (前十10個字元  每一個字元用 55個0/1 碼表示)
print('y[0]', y[0]*1)   ## show 出第一筆輸出變數y 資料!! (第十一個字元  該字元用 55個0/1 碼表示)



input_chars: ['project gu', 'roject gut', 'oject gute', 'ject guten', 'ect gutenb', 'ct gutenbe', 't gutenber', ' gutenberg', 'gutenbergs', 'utenbergs ', 'tenbergs a', 'enbergs al', 'nbergs ali', 'bergs alic', 'ergs alice', 'rgs alices', 'gs alices ', 's alices a', ' alices ad', 'alices adv', 'lices adve', 'ices adven', 'ces advent', 'es adventu', 's adventur', ' adventure', 'adventures', 'dventures ', 'ventures i', 'entures in', 'ntures in ', 'tures in w', 'ures in wo', 'res in won', 'es in wond', 's in wonde', ' in wonder', 'in wonderl', 'n wonderla', ' wonderlan', 'wonderland', 'onderland,', 'nderland, ', 'derland, b', 'erland, by', 'rland, by ', 'land, by l', 'and, by le', 'nd, by lew', 'd, by lewi', ', by lewis', ' by lewis ', 'by lewis c', 'y lewis ca', ' lewis car', 'lewis carr', 'ewis carro', 'wis carrol', 'is carroll', 's carroll ', ' carroll t', 'carroll th', 'arroll thi', 'rroll this', 'roll this ', 'oll this e', 'll this eb', 'l this ebo', ' this eboo', 'this ebook', 'his e

In [10]:
# build our model
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
## input_shape=  (10(列), 55(行))
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,input_shape=(SEQLEN, nb_chars),unroll=True))  
model.add(Dense(nb_chars))  ## 配合輸出變數y 資料!! (55個神經元)
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")
print(model.summary())

##  return_sequences 意思是在每個時間點，要不要輸出output，默認的是 false。
## 如果等於 false，就是只在最後一個時間點輸出一個值。
##  stateful，默認的也是 false，意義是批和批之間是否有聯繫。直觀的理解就是我們在讀完二十步，
## 第21步開始是接著前面二十步的。也就是第一個 batch中的最後一步與第二個 batch 中的第一步之間是有聯繫的。
## unroll：布林值，默認為False，若為True，則迴圈層將被展開，否則就使用符號化的迴圈。當使用TensorFlow為後端時，迴圈網路本來就是展開的，
## 因此該層不做任何事情。層展開會佔用更多的記憶體，但會加速RNN的運算。層展開只適用於短序列。

NUM_ITERATIONS=25
NUM_EPOCHS_PER_ITERATION=1
                                                            
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)   ## 這樣的訓練方式跟直接epoch多一些  有何差異
    test_idx = np.random.randint(len(input_chars))      ## len(input_chars)=158773筆資料中  產生一個筆號
    test_chars = input_chars[test_idx]					##  從訓練資料中  抓出該筆資料
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):  ## 動態(添加預測資料做為 input) 往前預測NUM_PREDS_PER_EPOCH筆!!
        Xtest = np.zeros((1, SEQLEN, nb_chars))   ## 產生一組  列=SEQLEN=10, 行= nb_chars=55   值都是 0 的矩陣資料
        for i, ch in enumerate(test_chars):   ## list(enumerate(test_chars))  可以看!!    這一邊厲害  i ch 是跟著enumerate(test_chars)  動!!
            Xtest[0, i, char2index[ch]] = 1   ##  把出現的字母依照其位置令其值為 1   很厲害
        pred = model.predict(Xtest, verbose=0)[0]  ##  加了後面的[0] 可以把[1,55] 變成 [55,]   預測各行的發生機率
		## verbose 日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄
        ypred = index2char[np.argmax(pred)]  ## np.argmax(pred)  找機率最大值
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred    ## 把預測值加上去  變成新的測試資料   開始往下繼續預測!!
    print('\r\n')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               23552     
_________________________________________________________________
dense_4 (Dense)              (None, 55)                7095      
_________________________________________________________________
activation_1 (Activation)    (None, 55)                0         
Total params: 30,647
Trainable params: 30,647
Non-trainable params: 0
_________________________________________________________________
None
Iteration #: 0
Epoch 1/1
158773/158773 [==============================] - 5s 32us/step - loss: 2.3341
Generating from seed: and then t
and then the the said the said the said the said the said the said the said the said the said the said the sai

Iteration #: 1
Epoch 1/1
158773/158773 [==============================] - 5s 29us/step - loss: 2.0495
Generating from seed: ly bowed, 
ly bowed, and t

# sonnets

In [11]:

# coding: utf-8

# ## Importing Dependencies

import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RNN
from keras.utils import np_utils


# ## Loading of Data

text = (open("D:/Value of Dream/Marathon_Github/NLP/data/sonnets.txt").read())
text=text.lower()

# ## Creating character/word mappings

characters = sorted(list(set(text)))

n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}


# ## Data pre-processing

X = []
Y = []
length = len(text)
seq_length = 100  ## 定義要用多少個前面的字元來預測

## 下面開始切割資料  X 與 y
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    X.append([char_to_n[char] for char in sequence])
    Y.append(char_to_n[label])

X_modified = np.reshape(X, (len(X), seq_length, 1))  ## 定義 shape for Model
X_modified = X_modified / float(len(characters))    ## 壓縮資料在 0-1 之間
Y_modified = np_utils.to_categorical(Y)



In [ ]:

# ## A baseline model

model = Sequential()
model.add(LSTM(400, input_shape=(X_modified.shape[1], X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(X_modified, Y_modified, epochs=1, batch_size=100)

model.save_weights('d://text_generator_400_0.2_400_0.2_baseline.h5')

model.load_weights('d://text_generator_400_0.2_400_0.2_baseline.h5')


# ## Generating Text

string_mapped = X[99]
full_string = [n_to_char[value] for value in string_mapped]


Epoch 1/1
46600/97820 [=============>................] - ETA: 19:27 - loss: 3.0076

In [ ]:
# generating characters
for i in range(400):  ## 從第100 個字(X[99])開始  往下一步一步預測400 個字元
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))

    pred_index = np.argmax(model.predict(x, verbose=0)) ##預測的是索引值
    seq = [n_to_char[value] for value in string_mapped]
    full_string.append(n_to_char[pred_index])  ##將索引值轉成字元!! 連接起來

    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

#combining text
txt=""
for char in full_string:
    txt = txt+char
txt
